# Ayudantía Control 2

Miguel Martínez

## Esquema

### Stores

`id`: <tt>int primary key</tt> \
`email`: <tt>varchar not null</tt> \
`name`: <tt>varchar not null</tt> \
`created_at`: <tt>date not null</tt> 

### Products

`id`: <tt>int primary key</tt>\
`name`: <tt>varchar not null</tt>\
`store_id`: <tt>int not null, foreign key from `Stores`</tt>\
`row_order`: <tt>int</tt>\
`created_at`: <tt>date not null</tt>

### Variants

`id`: <tt>int primary key</tt>\
`product_id`: <tt>int not null, foreign key from `Products`</tt>\
`is_master`: <tt>varchar not null, 'True' or 'False'</tt>\
`price`: <tt>int not null</tt>\
`quantity`: <tt>int not null</tt>\
`name_suffix`: <tt>varchar not null</tt>\
`created_at`: <tt>date not null</tt>

### Bills

`id`: <tt>int primary key</tt>\
`email`: <tt>varchar not null</tt>\
`address`: <tt>varchar not null</tt>\
`store_id`: <tt>int not null, foreign key from `Stores`</tt>\
`status`: <tt>varchar not null</tt>\
`created_at`: <tt>date not null</tt>

### BillVariants

`id`: <tt>int primary key</tt>\
`bill_id`: <tt>int not null, foreign key from `Bills`</tt>\
`quantity`: <tt>int not null</tt>\
`price`: <tt>int not null</tt>\
`variant_id`: <tt>int not null, foreign key from `Variants`</tt>\
`discount_rate`: <tt>int default(0)</tt>\
`created_at`: <tt>date not null</tt>


## Setup

In [ ]:
%load_ext sql

In [ ]:
%sql sqlite:///tu_shop_mini.db

In [ ]:
%sql SELECT COUNT(*) FROM Bills;

In [ ]:
%%sql
SELECT * FROM Products
LIMIT 1;

## Consultas

1. Los nombres de productos que sean vendidos por la tienda `tío zetamart` y la tienda `pc_faketory`. Un producto se considerará igual a otro si ambos tienen el mismo nombre.

In [ ]:
%%sql
SELECT Products.name
FROM Products, Stores
WHERE Products.store_id = Stores.id AND Stores.name = 'tío zetamart'
INTERSECT
SELECT Products.name
FROM Products, Stores
WHERE Products.store_id = Stores.id AND Stores.name = 'pc_faketory'

2. La cantidad de unidades vendidas de cada producto en cada tienda.

In [ ]:
%%sql
SELECT Stores.name, Products.name, SUM(BillVariants.quantity)
FROM Stores, Products, Billvariants, Variants
WHERE Stores.id = Products.store_id
AND Variants.product_id = Products.id
AND BillVariants.variant_id = Variants.id
GROUP BY Stores.id, Products.id

3. El valor de las ventas de cada tienda en marzo, ordenadas por ventas.

In [ ]:
%%sql
SELECT Stores.name, SUM(BillVariants.quantity * BillVariants.price * ((100 - BillVariants.discount_rate)*1.0/100)) as ventas
FROM Stores, Products, Variants, BillVariants
WHERE Stores.id = Products.store_id
AND Variants.product_id = Products.id
AND BillVariants.variant_id = Variants.id
AND BillVariants.created_at BETWEEN '2021-03-01' AND '2021-04-01'
GROUP BY Stores.id
ORDER BY ventas

4. Los *ids* de las variantes que no han tenido ninguna venta en marzo y tienen stock disponible.

In [ ]:
%%sql
SELECT Variants.id
FROM Variants
WHERE Variants.quantity > 0
EXCEPT
SELECT Variants.id
FROM Variants, BillVariants
WHERE Variants.id = BillVariants.variant_id
AND BillVariants.created_at BETWEEN '2021-03-01' AND '2021-04-01'

5. El nombre y precio de la variante con el precio más caro.

In [ ]:
%%sql
SELECT (Products.name || ' ' || Variants.name_suffix) as full_name, Variants.price
FROM Products, Variants
WHERE Products.id = Variants.product_id
AND Variants.price = (SELECT MAX(Variants.price) FROM Variants)

6. El valor promedio de venta de cada tienda en marzo.

In [ ]:
%%sql
SELECT Stores.name, AVG(ventas_marzo.ventas)
  FROM Stores JOIN ( SELECT Stores.id as store_id, SUM(BillVariants.quantity * BillVariants.price * ((100 - BillVariants.discount_rate)*1.0/100)) as ventas
  FROM Stores, Products, Variants, BillVariants
  WHERE Stores.id = Products.store_id
  AND Variants.product_id = Products.id
  AND BillVariants.variant_id = Variants.id
  AND BillVariants.created_at BETWEEN '2021-03-01' AND '2021-04-01'
  GROUP BY Stores.id
  ) as ventas_marzo
  WHERE Stores.id = ventas_marzo.store_id
GROUP BY Stores.id

7. El porcentaje del total de clientes que fueron recurrentes en el mes de abril. Se consideran recurrentes a aquellos que hayan comprado alguna vez antes de comprar en abril. Los clientes se identifican por el mail.

In [ ]:
%%sql
Select COUNT(DISTINCT clientes_abril.email)*1.0/COUNT(DISTINCT Bills.email)*100 as porcentaje
FROM Bills LEFT OUTER JOIN (
  SELECT Bills.email
  FROM Bills
  WHERE Bills.created_at < '2021-04-01'
) as clientes_abril
ON Bills.email = clientes_abril.email
AND Bills.created_at > '2021-04-01'

8. La cantidad de días en promedio que han pasado desde que se creó cada venta, agrupada por <tt>status</tt>.



*   `DATE(SUBSTR(column, 0, 11))` retorna una `DATE` de sqlite a partir de una columna de fecha de nuestra base de datos.
*   `DATE('now')` retorna un objeto `DATE` de sqlite con la fecha de hoy.
*   La función `JULIANDAY(date)` retorna los días que han pasado entre `date` y una fecha constante muy en el pasado.



In [ ]:
%%sql
SELECT Bills.status, AVG(JULIANDAY(DATE('now')) - JULIANDAY(DATE(SUBSTR(Bills.created_at, 0, 11)))) AS average_diff
FROM Bills
GROUP BY Bills.status

9. El nombre de la tienda que más ventas hizo en abril, junto con la cantidad de ventas.

Sin `LIMIT`

In [ ]:
%%sql
SELECT Stores.name, COUNT(Bills.id) as numero_ventas
FROM Stores, Bills
WHERE Stores.id = Bills.store_id
AND Bills.created_at BETWEEN '2021-04-01' AND '2021-05-01'
GROUP BY Stores.id
HAVING numero_ventas = (SELECT MAX(aux.total_bills)
  FROM (
    SELECT COUNT(Bills.id) as total_bills
    FROM Stores JOIN Bills
    ON Stores.id = Bills.store_id
    WHERE Bills.created_at BETWEEN '2021-04-01' AND '2021-05-01'
    GROUP BY Stores.id
  ) as aux
);

Con `LIMIT`

In [ ]:
%%sql
SELECT  Stores.name, COUNT(bills.id) as sales_count
FROM Stores JOIN Bills
ON Stores.id = Bills.store_id
WHERE Bills.created_at BETWEEN '2021-04-01' AND '2021-05-01'
GROUP BY Stores.id
ORDER BY sales_count DESC
LIMIT 5